In [195]:
import pandas as pd
import numpy as np
import re

# Table of contents

1. [Introduction](#introduction)
2. [Webscrapping](#webscrapper)
3. [Preprocessing Webscrapperdata](#preprocessing_web)
4. [Geocoding with GeoAdminAPI](#geocoding)
5. [Append external data](#geodata)
6. [Write data to MySQL database](#MySQL)


## Introduction <a name="introduction"></a>
This ipynb file scrapes Data about Apartment advertisements in Zürich and enriches the Data by encoding the longitude and latitude data with the API form GeoAPI?. Further it enriches the Dataset with information about the nearest hospital, nearest public transport stop and information about the road noise emissions. Additonally information about the year when the building was built. All these additional Datasets have been downloaded form the Stadt Zürich website. Finally the dataset is uploaded to a MySQL database.

## Webscrapping <a name="webscrapper"></a>


In [196]:
# Libraries
import os
import re
import json
import time
import random
import pandas as pd

from bs4 import BeautifulSoup

from prettytable import from_csv

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(f'Current working directory: {os.getcwd()}')

# Liste mit User-Agents für Rotation
ua_path = "user_agents.txt"
ua_list = [line.rstrip('\n') for line in open(ua_path)]
ua_list[:5]

Current working directory: c:\Users\gigim\Documents\GitHub\mlp\mlp


['Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/37.0.2062.94 Chrome/37.0.2062.94 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/600.8.9 (KHTML, like Gecko) Version/8.0.8 Safari/600.8.9']

#### Install ChromeWebdriver and specity the folder where the dirver is located
https://googlechromelabs.github.io/chrome-for-testing/#stable
The file has to be executed before reading it as service object, windows blocks the execution of the program the first time and maybe a signoput or restart of the pc is needeed if it still cant access the chromedriver.

In [197]:
import random
# specify path of Chromedriver Location:
chromedriver_path = r'C:\Users\gigim\Documents\chromedriver-win64\chromedriver.exe'

# Set options for Chrome-Driver (headless mode)
opts = Options()
opts.add_argument("--headless")
opts.add_argument("user-agent=" + random.choice(ua_list))

service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=opts)

In [198]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options #for user Agents



page = 1
first_page = True
df = pd.DataFrame()

#iterate thriugh pages
while True:
    if page < 50:
        driver = webdriver.Chrome(service=service, options=opts)
        opts.add_argument("user-agent=" + random.choice(ua_list))
        driver = webdriver.Chrome(service=service, options=opts)
        url = f'https://www.immoscout24.ch/de/immobilien/mieten/plz-8000-zuerich?pn={page}'

        # Load the page
        driver.get(url)
        
        # Get the full content of the website
        source = driver.page_source

        # Close driver
        driver.close()

        # Parse HTML content with BeautifulSoup
        soup = BeautifulSoup(source, 'html.parser')
        #print('soup:', soup)


        #code for scrapping data -----------------------------------------------------------------------------------
        # Get links
        links = []
        link_elements = soup.find_all(class_='HgCardElevated_content_uir_2 HgCardElevated_link_EHfr7')
        for element in link_elements:
            print(element['href'])
            links.append(f'https://www.immoscout24.ch/de/immobilien/mieten/plz-8000-zuerich?pn={page}' + element['href'])
            #class2: HgCardElevated_content_uir_2 HgCardElevated_link_EHfr7
            #class_='HgListingCard_info_RKrwz'
        # Get types
        types_elements = soup.find_all(class_='HgListingRoomsLivingSpacePrice_roomsLivingSpacePrice_M6Ktp') #zimmer 
        types = [element.text for element in types_elements]

        # Get prices
        prices_elements = soup.find_all(class_='HgListingRoomsLivingSpacePrice_price_u9Vee')
        prices = [element.text for element in prices_elements]


        # Get address
        address_elements = soup.find_all(class_='HgListingCard_address_JGiFv')
        address = [element.text for element in address_elements]


        # Get description
        description_elements = soup.find_all(class_='HgListingDescription_title_NAAxy')
        description = [element.text for element in description_elements]

        # Get description small
        #descriptionSmall_elements = soup.find_all(class_='HgListingDescription_large_uKs3J')
        #descriptionSmall = [element.text for element in descriptionSmall_elements]

        # Check lenght of the datasets
        print("Types:", len(types))
        print("Prices:", len(prices))
        print("Addresses:", len(address))
        print("Descriptions:", len(description))
        #print("DescriptionSmall:", len(descriptionSmall))
        print("Links:", len(links))

        # Dataframe
        df1 = pd.DataFrame({ 'Page': page,
                            'Type': types,
                            'Price': prices,
                            'Address': address,
                            'Description': description,
                            #'DescriptionSmall': descriptionSmall, #funktioniert nicht da nur 12 von 20 inserate dies haben, werte können nicht den dazugehörigen inseraten zugeweisen werden
                            'Weblink': links})
        #Save to file
        #df.to_csv('autoscout24.csv', sep="|", index=False, encoding='utf-8')

        # Append the temporary DataFrame to the main DataFrame
        df = pd.concat([df, df1], ignore_index=True)
        #end scrapping data -----------------------------------------------------------
        url = f'https://www.immoscout24.ch/de/immobilien/mieten/plz-8000-zuerich?pn={page}'
        print(url)
        page = page + 1
    else:
        break

/mieten/4001022281
/mieten/4000809249
/mieten/4000862528
/mieten/4000857089
/mieten/3003373030
/mieten/4001092487
/mieten/4000598194
/mieten/4000896027
/mieten/4001055744
/mieten/4001022282
/mieten/4000898655
/mieten/4001082533
/mieten/4001082202
/mieten/4001067246
/mieten/4001092488
/mieten/4000857261
/mieten/4000671436
/mieten/4001088528
/mieten/4001088524
/mieten/4001088513
Types: 20
Prices: 20
Addresses: 20
Descriptions: 20
Links: 20
https://www.immoscout24.ch/de/immobilien/mieten/plz-8000-zuerich?pn=1
/mieten/4001088491
/mieten/4001097767
/mieten/4001041701
/mieten/4001084385
/mieten/4001068921
/mieten/4001067330
/mieten/4001077906
/mieten/4001077903
/mieten/4001090524
/mieten/4001032379
/mieten/4001081748
/mieten/4001086144
/mieten/4001079388
/mieten/4001079387
/mieten/4001021207
/mieten/4001023502
/mieten/4000816406
/mieten/4001101201
/mieten/4001100453
/mieten/4001077065
Types: 20
Prices: 20
Addresses: 20
Descriptions: 20
Links: 20
https://www.immoscout24.ch/de/immobilien/miete

In [199]:
# 4 testing
df_copy = df.copy()

## Preprocessing Webscrapperdata <a name="preprocessing_web"></a>

In [237]:
# 4 testing
df = df_copy.copy()

In [238]:
df.columns

Index(['Page', 'Type', 'Price', 'Address', 'Description', 'Weblink'], dtype='object')

In [239]:
# Attribute Type: Function to extract values from 'Type' attribute and store in new columns
def extract_values(row):
    parts = re.split(',\s|,\xa0', row['Type'])  # Split by comma followed by space or non-breaking space
    print(parts)
    rooms = None
    size = None
    price2 = None
    if len(parts) == 1:
        price2 = parts[0]
    if len(parts) == 2:
        size = parts[0]
        price2 = parts[1]
    elif len(parts) == 3:
        rooms = parts[0]
        size = parts[1]
        price2 = parts[2]
    return pd.Series([rooms, size, price2], index=['rooms', 'size', 'price2'])

# Apply the function to each row of the DataFrame
df[['rooms', 'size', 'price2']] = df.apply(extract_values, axis=1)


# Function to move 'size' values containing 'Zimmer' to 'rooms' attribute
def move_to_rooms(row):
    if row['size'] is not None and 'zimmer' in row['size'].lower():
        row['rooms'], row['size'] = row['size'], None
    return row

# Apply the function to each row of the DataFrame
df = df.apply(move_to_rooms, axis=1)


print('Split values form Type:')
print(df)
nan_count = df['size'].isna().sum()
print('missing size:', nan_count)
df.dropna(subset=['size'], inplace=True)
# Remove 'm²' and any other non-digit characters (including commas, spaces, and special apostrophes)
df['size'] = df['size'].str.replace('m²', '')
df['size'] = df['size'].str.replace('[^0-9]', '', regex=True)
df['size'] = df['size'].astype(int)


['4.5 Zimmer', '114m²', 'CHF 4’032.– ']
['2.5 Zimmer', '90m²', 'CHF 3’220.– ']
['3.5 Zimmer', '107m²', 'CHF 3’600.– ']
['1.5 Zimmer', '24m²', 'CHF 1’720.– ']
['2.5 Zimmer', '54m²', 'CHF 2’600.– ']
['4.5 Zimmer', '105m²', 'CHF 4’090.– ']
['1.5 Zimmer', '26m²', 'CHF 1’921.– ']
['3.5 Zimmer', '180m²', 'CHF 4’835.– ']
['1 Zimmer', '18m²', 'CHF 1’380.– ']
['4.5 Zimmer', '117m²', 'CHF 4’227.– ']
['4.5 Zimmer', '117m²', 'CHF 4’262.– ']
['2.5 Zimmer', '60m²', 'CHF 3’710.– ']
['2.5 Zimmer', '77m²', 'CHF 3’240.– ']
['5.5 Zimmer', '113m²', 'CHF 4’720.– ']
['3.5 Zimmer', '86m²', 'CHF 3’850.– ']
['1.5 Zimmer', '45m²', 'CHF 2’730.– ']
['4 Zimmer', '125m²', 'CHF 4’390.– ']
['3 Zimmer', '70m²', 'CHF 3’950.– ']
['2 Zimmer', '40m²', 'CHF 2’950.– ']
['1.5 Zimmer', '55m²', 'CHF 2’650.– ']
['2 Zimmer', '40m²', 'CHF 2’750.– ']
['4.5 Zimmer', '154m²', 'CHF 5’450.– ']
['2 Zimmer', '55m²', 'CHF 1’755.– ']
['7.5 Zimmer', '205m²', 'CHF 5’695.– ']
['5.5 Zimmer', '127m²', 'CHF 6’117.– ']
['4.5 Zimmer', '145m²', 'C

In [240]:
# Attribute Rooms: One hot encoding

# Function to check and update 'rooms' attribute based on 'Description'
def update_rooms(row):
    if pd.isna(row['rooms']):
        description = row['Description'].lower()
        if 'studio' in description:
            row['rooms'] = '1 Zimmer'
        elif '1 bedroom' in description:
            row['rooms'] = '1 Zimmer'
        elif '2 bedroom' in description:
            row['rooms'] = '2 Zimmer'
    return row

# Apply the function to each row of the DataFrame
df = df.apply(update_rooms, axis=1)


df.dropna(subset=['rooms'], inplace=True)

to be used later when

 Filter out None values from 'rooms' attribute
filtered_rooms = [room for room in df['rooms'] if room is not None]

 Extract unique values
unique_rooms = list(set(filtered_rooms))

 Sort unique values
unique_rooms.sort()

 Create new attributes for each unique value
for room_type in unique_rooms:
    df[room_type] = df['rooms'].apply(lambda x: 1 if x == room_type else 0)

 Remove the word 'Zimmer' from the 'rooms' column and convert to numeric
     Value will be usefull for EDA
df['rooms'] = df['rooms'].str.replace(' Zimmer', '').astype(float)

 Filter out rows where the sum of these specific columns is zero (might have to be moved up)
df = df[df[unique_rooms].sum(axis=1) > 0]



In [241]:
# Attribute Address: 

# Replace 'str.' with 'strasse' in the 'address' column
df['Address'] = df['Address'].str.replace('str\.', 'strasse', regex=True)

# Custom function to edit addresses: remove values with additional comma
def edit_address(Address):
    # Split the address by commas
    parts = Address.split(', ')
    
    # Check if there are two commas
    if len(parts) == 3:
        # Join the parts at positions 0 and 2 to remove the part at position 1
        edited_address = ', '.join([parts[0], parts[2]])
        return edited_address
    
    # Return the original address if there are not two commas
    return Address

# Apply the function to edit addresses in the 'address' column
df['Address'] = df['Address'].apply(edit_address)

# 'Address' into 'address_short_1' and 'address_short_2' with conditions
def split_address(row):
    parts = row['Address'].split(', ')
    if len(parts) == 2:
        row['address_short_1'] = parts[0]
        row['address_short_2'] = parts[1]
    elif len(parts) == 1:
        row['address_short_1'] = None
        row['address_short_2'] = parts[0]
    return row

# Apply the function to each row of the DataFrame
df = df.apply(split_address, axis=1)

# Drop rows with '/' or None values and where the last character is not a digit
df = df[df['address_short_1'].notna() & (df['address_short_1'].str[-1].str.isdigit())]
# Remove 'address_short_1' and 'address_short_2' columns
df = df.drop(columns=['address_short_1', 'address_short_2'])

In [242]:
# Attribute Price: convert to int
for index, row in df.iterrows():
    price_str = df.at[index, 'Price'].strip()
    # Remove "CHF" and any non-numeric characters, replace thousands separator

    cleaned_str = re.sub(r'[^\d]', '', price_str.replace("’", ''))

    # Convert to integer
    try:
        df.at[index, 'Price'] = int(cleaned_str)
    except:
        df.at[index, 'Price'] = np.NAN

nan_count = df['Price'].isna().sum()
print(nan_count)
df.dropna(subset=['Price'], inplace=True)

# To verify if all have been removed
df[df['Price'].isna()]

0


,Page,Type,Price,Address,Description,Weblink,rooms,size,price2


In [243]:
df

,Page,Type,Price,Address,Description,Weblink,rooms,size,price2
0,1,"4.5 Zimmer, 114m², CHF 4’032.–",4032,"Käferholzstrasse 42, 8057 Zürich",Modernes Wohnen im Guggachpark,https://www.immoscout24.ch/de/immobilien/miete...,4.5 Zimmer,114,CHF 4’032.–
1,1,"2.5 Zimmer, 90m², CHF 3’220.–",3220,"Grabenwies 2, 8057 Zürich",Ihre neue Traumwohnung!,https://www.immoscout24.ch/de/immobilien/miete...,2.5 Zimmer,90,CHF 3’220.–
3,1,"1.5 Zimmer, 24m², CHF 1’720.–",1720,"Herrligweg 9, 8048 Zürich",Zentrale 1.5 Zimmerwohnung,https://www.immoscout24.ch/de/immobilien/miete...,1.5 Zimmer,24,CHF 1’720.–
4,1,"2.5 Zimmer, 54m², CHF 2’600.–",2600,"Hagenholzstrasse 59, 8050 Zürich",One-bedroom apartment in the heart of Zurich,https://www.immoscout24.ch/de/immobilien/miete...,2.5 Zimmer,54,CHF 2’600.–
5,1,"4.5 Zimmer, 105m², CHF 4’090.–",4090,"Widmerstrasse 87, 8038 Zürich",Erstvermietung - Schöner wohnen in Zürich-Süd:...,https://www.immoscout24.ch/de/immobilien/miete...,4.5 Zimmer,105,CHF 4’090.–
...,...,...,...,...,...,...,...,...,...
415,38,"3 Zimmer, 61m², CHF 2’290.–",2290,"Scheuchzerstrasse 20, 8006 Zürich",Befristetes Mietverhältnis in Altbauwohnung,https://www.immoscout24.ch/de/immobilien/miete...,3 Zimmer,61,CHF 2’290.–
416,38,"1.5 Zimmer, 42m², CHF 2’320.–",2320,"Ottenweg 11, 8008 Zürich",Wohnen im trendigen Seefeld-Quartier?,https://www.immoscout24.ch/de/immobilien/miete...,1.5 Zimmer,42,CHF 2’320.–
417,38,"1 Zimmer, 25m², CHF 2’450.–",2450,"Langstrasse 213, 8005 Zürich",Studio Apartment im Zentrum Zürichs,https://www.immoscout24.ch/de/immobilien/miete...,1 Zimmer,25,CHF 2’450.–
418,38,"3.5 Zimmer, 85m², CHF 2’953.–",2953,"In Böden 132, 8046 Zürich",Lichtdurchflutete Parterrewohnung m. Terrasse,https://www.immoscout24.ch/de/immobilien/miete...,3.5 Zimmer,85,CHF 2’953.–


In [244]:
df.to_clipboard()

In [245]:
# for at the end of preprocessing rename the collumns to lowercase

# Rename columns to lowercase
df.rename(columns=lambda x: x.lower(), inplace=True)



df = df.drop(columns=['page', 'type', 'weblink', 'price2'])

In [246]:
df_copy2 = df.copy()

## Geocoding with GeoAdminAPI <a name="geocoding"></a>

In [247]:
df= df_copy2.copy()

In [248]:
import pandas as pd
import numpy as np

import json

import urllib.parse

In [249]:
df_merged2.to_clipboard()

In [250]:
import pandas as pd
import requests

def geocode_addresses(address_list):
    base_url = "https://api3.geo.admin.ch/rest/services/api/SearchServer"
    geocoded_results = []

    for address in address_list:
        params = {
            'searchText': address,
            'type': 'locations',
            'sr': '4326',  # Use WGS84 for global standard latitude and longitude
            'returnGeometry': 'true'
        }
        
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            results = response.json().get('results', [])
            if results:  # Check if there are any results
                first_result = results[0]  # Take only the first result
                attrs = first_result.get('attrs', {})
                geocoded_results.append({
                    'address': address,
                    'latitude': attrs.get('y'),
                    'longitude': attrs.get('x'),
                    'detail': attrs.get('detail')
                })
            else:
                geocoded_results.append({
                    'address': address,
                    'error': 'No results found'
                })
        else:
            geocoded_results.append({
                'address': address,
                'error': 'Failed to geocode'
            })

    return geocoded_results

# Assuming 'df' is your original DataFrame and has a column 'address'
addresses = df['address'].tolist()

# Get the geocoded data
geocoded_results = geocode_addresses(addresses)
df_geocoded = pd.DataFrame(geocoded_results)

# Merge the original DataFrame with the geocoded results
df = pd.merge(df, df_geocoded, on='address', how='left')

# Remove values with error message
df.drop(df[df['error'] == 'No results found'].index, inplace=True)

# Remove columns
df = df.drop(columns=['detail', 'error'])

# Remove duplicate rows
df = df.drop_duplicates()

# Reset the index
df.reset_index(drop=True, inplace=True)


In [251]:
df.to_clipboard()

In [252]:
## backup
df_copy4 = df.copy()

## Append external data <a name="geodata"></a>
Quelle: Stadt Zürich (https://data.stadt-zuerich.ch/dataset?tags=geodaten)


In [254]:
# Read Data (size of the datasets has been reduced to only the nessesairy data)
df_spital = pd.read_csv('geodata/df_spital.csv') # Hospitals in Zürich
#df_laerm = pd.read_csv('geodata/df_laerm.csv') # Sound Emmissions in Zürich
df_ov = pd.read_csv('geodata/df_ov.csv') # Public transport stops
#df_year = pd.read_csv('geodata/df_register.csv') # Buildting construction date

import gzip
import pandas as pd

# Read data from GZIP file
gzip_file_1 = 'geodata/df_laerm.csv.gz'
with gzip.open(gzip_file_1, 'rb') as f:
    df_laerm = pd.read_csv(f) # Sound Emmissions in Zürich

gzip_file_2 = 'geodata/df_register.csv.gz'
with gzip.open(gzip_file_2, 'rb') as f:
    df_year = pd.read_csv(f) # Buildting construction date



In [255]:
df_24 = df.copy()

In [256]:
from scipy.spatial import cKDTree
import haversine as hs
import pandas as pd

def calculate_distance(df_add, df_24, distance_type: str, laerm=False):
    # Ensure the dataframes have the correct index type
    df_add = df_add.reset_index(drop=True)
    df_24 = df_24.reset_index(drop=True)
    
    # Convert DataFrame columns to numpy arrays for KD-Tree
    tree = cKDTree(df_add[['latitude', 'longitude']].to_numpy())
    distances, indices = tree.query(df_24[['latitude', 'longitude']].to_numpy(), k=1)

    # Pre-compute the locations as a list of tuples for efficiency
    df_24_locs = list(df_24[['latitude', 'longitude']].itertuples(index=False, name=None))
    df_add_locs = list(df_add[['latitude', 'longitude']].itertuples(index=False, name=None))

    # If laerm is False, calculate and store the haversine distances
    if not laerm:
        df_24[distance_type] = [hs.haversine(df_24_locs[i], df_add_locs[indices[i]]) for i in range(len(indices))]
    else:
        # If laerm is True, transfer 'lr_day' and 'lr_night' from df_add to df_24
        df_24['lr_day'] = [df_add.iloc[indices[i]]['lr_day'] for i in range(len(indices))]
        df_24['lr_night'] = [df_add.iloc[indices[i]]['lr_night'] for i in range(len(indices))]
    
    return df_24

In [257]:
df_24 = calculate_distance(df_spital, df_24, 'distance_hospital')
df_24 = calculate_distance(df_laerm, df_24, 'distance_laerm', True)
df_24 = calculate_distance(df_ov, df_24, 'distance_ov')

df_year = df_year.drop_duplicates(subset='address', keep='first')
# Merge Constructon year and df on address
df_24 = pd.merge(df_24, df_year, on='address',  how='left')
df_24['wbauj'].fillna(0, inplace=True)

## Write data to MySQL database <a name="MySQL"></a>

In [258]:
import mysql.connector #pip install mysql-connector-python

# Replace these with your connection parameters
config = {
    'host': '127.0.0.1',
    'port':3306,
    'user': 'root',
    'password': 'GabaKDube731',
    'database': 'mlp'
}

# Connect to the MySQL Database
try:
    conn = mysql.connector.connect(**config)
    if conn.is_connected():
        print('Successfully connected to the database')
except mysql.connector.Error as e:
    print('Error:', e)

Successfully connected to the database


In [261]:
# Verify that collumns match the target table
df_24.columns

Index(['price', 'address', 'description', 'rooms', 'size', 'latitude',
       'longitude', 'distance_hospital', 'lr_day', 'lr_night', 'distance_ov',
       'wbauj'],
      dtype='object')

In [260]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

# SQL statement to create the table if it does not exist
create_table_query = """
CREATE TABLE IF NOT EXISTS dta (
    price INT,
    address VARCHAR(255),
    description TEXT,
    rooms VARCHAR(255),
    size VARCHAR(255),
    latitude DOUBLE,
    longitude DOUBLE,
    distance_hospital DOUBLE,
    lr_day FLOAT,
    lr_night FLOAT,
    distance_ov DOUBLE,
    wbauj INT
);
"""

# Execute the create table query
execute_query(conn, create_table_query)

Query executed successfully


In [269]:
# function to upload data to SQL table
def dataframe_to_sql(connection, df, table_name):
    cursor = connection.cursor()
    for _, row in df.iterrows():
        query = f"""
        INSERT INTO {table_name} (price, address, description, rooms, size, latitude, longitude, distance_hospital, lr_day, lr_night, distance_ov, wbauj)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = (row['price'], row['address'], row['description'],
                  row['rooms'], row['size'],
                  row['latitude'], row['longitude'], row['distance_hospital'],
                  row['lr_day'], row['lr_night'], row['distance_ov'], row['wbauj'])
        try:
            cursor.execute(query, values)
            connection.commit()
            print("Query executed successfully")
        except Error as e:
            print(f"The error '{e}' occurred")
            connection.rollback()

# function to read and create a DataFrame
def read_table_to_dataframe(connection, table_name):
    # example usage: df_temp = read_table_to_dataframe(conn, 'dta')
    query = f"SELECT * FROM {table_name}"
    try:
        return pd.read_sql(query, connection)
    except Error as e:
        print(f"The error '{e}' occurred")
        return None

In [271]:
# check for dublicate values
df_temp = read_table_to_dataframe(conn, 'dta')

# Conditional check
if not df_temp.empty:
    # Create a backup of df_24 before removing duplicates
    df_24_backup = df_24.copy()

    # Create a key for duplicate checking
    df_temp['check_dup'] = df_temp['address'] + '|' + df_temp['size'].astype(str)
    df_24['check_dup'] = df_24['address'] + '|' + df_24['size'].astype(str)

    # Remove duplicates by checking if 'check_dup' from df_24 exists in df_temp
    df_24 = df_24[~df_24['check_dup'].isin(df_temp['check_dup'])]

    # Optionally, drop the 'check_dup' column if no longer needed
    df_24.drop('check_dup', axis=1, inplace=True)
    df_temp.drop('check_dup', axis=1, inplace=True)

    # Display the first few rows to verify
    print(df_24.head())
    print("DataFrame is not empty. Removed duplicates from df_24.")
else:
    # Code to execute if the DataFrame is empty
    print("DataFrame is empty. No need to check for duplicates.")

Empty DataFrame
Columns: [price, address, description, rooms, size, latitude, longitude, distance_hospital, lr_day, lr_night, distance_ov, wbauj]
Index: []
DataFrame is not empty. Removed duplicates from df_24.


In [270]:
dataframe_to_sql(conn, df_24, 'dta')

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe